## 1. Import Libraries and Setup

Import all necessary libraries and configure the environment.

In [1]:
# Standard libraries
import os
import json
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# RealCause imports
from data.sepsis import load_sepsis
from utils import to_data_format

# CDV utilities
from cdv_utils.generator_validation import (
    train_multiple_models, analyze_model_performance, select_best_model,
    validate_generator_quality, generate_synthetic_data, create_dataframe_from_synthetic_data
)
from cdv_utils.causal_modeling import (
    assign_variants_by_patterns, group_by_variants_with_filtered_columns,
    process_test_data_with_training_variants
)
from cdv_utils.experiment_runner import (
    run_multi_seed_experiment, analyze_experiment_results
)

# Global constants
INITIAL_SEED = 420
EXPERIMENT_SEED = 42
R2_THRESHOLD = 0.1
NUM_VARIANTS = 3

print("Environment setup complete!")

c:\Users\Guy\anaconda3\envs\ICPM_paper2025_2nd_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Environment setup complete!


## 2. Data Loading and Initial Exploration

Load the sepsis dataset and examine its basic properties.

In [2]:
# Load the sepsis dataset
print("Loading sepsis dataset...")
sepsis_data = load_sepsis(data_format='numpy', dataroot="datasets")

# Extract data components
if isinstance(sepsis_data, tuple):
    w, t, y = sepsis_data
else:
    w, t, y = sepsis_data["w"], sepsis_data["t"], sepsis_data["y"]

# Display basic dataset statistics
print(f"\nDataset dimensions:")
print(f"- Covariates (w): {w.shape}")
print(f"- Treatment (t): {t.shape}")
print(f"- Outcome (y): {y.shape}")

print(f"\nTreatment distribution:")
print(f"- Treatment (1): {np.sum(t == 1)} ({np.mean(t == 1)*100:.2f}%)")
print(f"- Control (0): {np.sum(t == 0)} ({np.mean(t == 0)*100:.2f}%)")

print(f"\nOutcome statistics:")
print(f"- Min: {y.min()}, Max: {y.max()}")
print(f"- Mean: {y.mean():.4f}, Std: {y.std():.4f}")

Loading sepsis dataset...
Loading sepsis dataset from datasets\sepsis_cases.csv

Dataset dimensions:
- Covariates (w): (810, 30)
- Treatment (t): (810,)
- Outcome (y): (810,)

Treatment distribution:
- Treatment (1): 72 (8.89%)
- Control (0): 738 (91.11%)

Outcome statistics:
- Min: 280.0, Max: 55704.0
- Mean: 7457.6469, Std: 6749.6825


In [3]:
# Load original CSV to get feature names
original_sepsis_df = pd.read_csv(os.path.join("datasets", "sepsis_cases.csv"))
w_cols = [col for col in original_sepsis_df.columns if col not in ['t', 'y', 'y0', 'y1', 'ite', 'variant']]

print(f"Feature columns ({len(w_cols)}):")
for i, col in enumerate(w_cols):
    print(f"  {i+1:2d}. {col}")

print(f"\nDataset ready for RealCause modeling!")

Feature columns (30):
   1. InfectionSuspected
   2. DiagnosticBlood
   3. DisfuncOrg
   4. SIRSCritTachypnea
   5. Hypotensie
   6. SIRSCritHeartRate
   7. Infusion
   8. DiagnosticArtAstrup
   9. Age
  10. DiagnosticIC
  11. DiagnosticSputum
  12. DiagnosticLiquor
  13. DiagnosticOther
  14. SIRSCriteria2OrMore
  15. DiagnosticXthorax
  16. SIRSCritTemperature
  17. DiagnosticUrinaryCulture
  18. SIRSCritLeucos
  19. Oligurie
  20. DiagnosticLacticAcid
  21. Hypoxie
  22. DiagnosticUrinarySediment
  23. DiagnosticECG
  24. Leucocytes
  25. CRP
  26. LacticAcid
  27. Leucocytes_2
  28. CRP_2
  29. LacticAcid_2
  30. LacticAcid_3

Dataset ready for RealCause modeling!


## 3. RealCause Generator Training and Validation

Train RealCause models to generate synthetic data and validate their quality through statistical tests.

### 3.1 Train RealCause Models

Train multiple RealCause models with different architectures and seeds to find the best generator.

In [4]:
# Set up training configuration
saveroot = "save/sepsis_model_organized"
training_seeds = [INITIAL_SEED]  # Can add more seeds: [420, 421, 422]

print("Training RealCause models...")
print(f"- Save directory: {saveroot}")
print(f"- Training seeds: {training_seeds}")

# Train multiple models
results, best_model_selector_dict = train_multiple_models(
    w, t, y, 
    w_cols=w_cols,
    saveroot=saveroot,
    seeds=training_seeds
)

print("\nModel training completed!")
print(f"Trained architectures: {list(results.keys())}")
print(f"Models saved to: {saveroot}")

Training RealCause models...
- Save directory: save/sepsis_model_organized
- Training seeds: [420]

Training medium architecture:
- Hidden layers: 3
- Hidden units: 128
- Activation: ReLU
n_train: 405	n_val: 81	n_test: 324
test_idxs:  (324,)


  0%|          | 0/400 [00:00<?, ?it/s]

Iteration 10 valid loss 2.418264389038086
saving best-val-loss model
Iteration 20 valid loss 2.4024510383605957
saving best-val-loss model


  0%|          | 2/400 [00:00<00:33, 11.76it/s]

Iteration 30 valid loss 2.385331630706787
saving best-val-loss model
Iteration 40 valid loss 2.3649020195007324
saving best-val-loss model


...

 29%|██▉       | 115/400 [00:31<00:52,  5.39it/s]

Iteration 1500: 0.07223530858755112 -1.2168684005737305
Iteration 1500 valid loss -0.28760242462158203


 29%|██▉       | 117/400 [00:33<01:20,  3.50it/s]

Iteration 1510 valid loss -0.2953440248966217
Iteration 1520 valid loss -0.2759971618652344
Iteration 1530 valid loss -0.3239732086658478
early stopping criterion reached. Ending experiment.
loading best-val-loss model (early stopping checkpoint)



Model training completed!
Trained architectures: ['medium']
Models saved to: save/sepsis_model_organized


### 3.2 Analyze Model Performance

Compare the performance of different models using statistical metrics.

In [5]:
# Analyze model performance
performance_df = analyze_model_performance(results, saveroot)

print("Model Performance Comparison:")
display(performance_df)

# Select best model
best_model = select_best_model(results, best_model_selector_dict, criterion="medium")
print(f"\nSelected best model: {best_model.__class__.__name__}")
print(f"Model architecture: medium (3 layers, 128 hidden units)")

Model Performance Comparison:


,Architecture,Seed,Univariate: Y KS p-value,Univariate: T KS p-value,Univariate: Y ES p-value,Univariate: T ES p-value,Univariate: Y Wasserstein,Univariate: T Wasserstein,Multivariate: Wasserstein1 p-value,Multivariate: Wasserstein2 p-value,Multivariate: kNN p-value,Multivariate: Energy p-value,Multivariate: Friedman-Rafsky p-value,ATE,Model Path
0,medium,seed_420,0.28183,0.952218,0.241516,0.879783,816.970534,0.015432,0.02,0.152,0.117,0.012,0.235,-2324.199307,save/sepsis_model_organized\medium_seed_420\mo...



Selected best model: TarNet
Model architecture: medium (3 layers, 128 hidden units)


## 4. Generate Synthetic Training Data

Generate synthetic training data with counterfactuals for causal inference experiments.

In [6]:
# Generate synthetic training data
print("Generating synthetic training data with counterfactuals...")

w_train, t_train, y0, y1 = generate_synthetic_data(
    best_model, seed=EXPERIMENT_SEED, dataset='train'
)

# Create formatted dataframe
w_samples_df = create_dataframe_from_synthetic_data(
    w_train, t_train, y0, y1, w_cols
)

print(f"\nGenerated synthetic training data:")
print(f"- Samples: {len(w_samples_df)}")
print(f"- Features: {len([col for col in w_samples_df.columns if col in w_cols])}")
print(f"- Treatment distribution: {dict(w_samples_df['t'].value_counts())}")

# Display sample
print("\nSample of generated data:")
display(w_samples_df.head())

print(f"\nITE statistics:")
print(f"- Mean ITE: {w_samples_df['ite'].mean():.6f}")
print(f"- Std ITE: {w_samples_df['ite'].std():.6f}")
print(f"- ATE: {w_samples_df['ite'].mean():.6f}")

Generating synthetic training data with counterfactuals...

Generated synthetic training data:
- Samples: 405
- Features: 30
- Treatment distribution: {0.0: 366, 1.0: 39}

Sample of generated data:


,InfectionSuspected,DiagnosticBlood,DisfuncOrg,SIRSCritTachypnea,Hypotensie,SIRSCritHeartRate,Infusion,DiagnosticArtAstrup,Age,DiagnosticIC,...,LacticAcid,Leucocytes_2,CRP_2,LacticAcid_2,LacticAcid_3,t,y0,y1,y,ite
0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,90.0,1.0,...,2.4,-1.0,-1.0,-1.0,-1.0,0.0,13062.310547,299.000000,13062.310547,-12763.310547
1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,55.0,1.0,...,1.3,-1.0,-1.0,-1.0,-1.0,0.0,14308.323242,299.000000,14308.323242,-14009.323242
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1438.318848,684.349976,1438.318848,-753.968872
3,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,90.0,1.0,...,3.2,-1.0,-1.0,-1.0,-1.0,0.0,15942.497070,1923.374390,15942.497070,-14019.123047
4,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,80.0,1.0,...,3.8,-1.0,-1.0,-1.0,-1.0,0.0,4340.021484,4846.141602,4340.021484,506.120117



ITE statistics:
- Mean ITE: -1987.729980
- Std ITE: 7365.526367
- ATE: -1987.729980


## 5. Variant Assignment and Data Preparation

Assign data to variants based on feature patterns and prepare datasets for causal modeling.

In [7]:
# Calculate feature patterns and assign variants
print("Assigning variants based on feature patterns...")

# Get feature columns
feature_cols = [col for col in w_samples_df.columns if col not in ['t', 'y', 'y0', 'y1', 'ite', 'variant', 'feature_pattern']]
feature_mask = w_samples_df[feature_cols] >= 0
# print(feature_mask)
w_samples_df['feature_pattern'] = feature_mask.apply(
    lambda row: ''.join(['1' if val else '0' for val in row]), axis=1
)

# Analyze pattern frequencies
pattern_counts = w_samples_df['feature_pattern'].value_counts()
print(f"\nFound {len(pattern_counts)} unique feature patterns")
print(f"Top 10 patterns:")
for i, (pattern, count) in enumerate(pattern_counts.head(10).items(), 1):
    print(f"  {i:2d}. Pattern {pattern}: {count} cases ({count/len(w_samples_df)*100:.2f}%)")

# Get top variants for assignment
k = NUM_VARIANTS
top_variants = pattern_counts.index[:k-1]  # Top k-1 patterns

print(f"\nUsing top {k-1} patterns as specific variants:")
for i, pattern in enumerate(top_variants, 1):
    count = pattern_counts[pattern]
    print(f"  Variant {i}: {pattern} ({count} cases, {count/len(w_samples_df)*100:.2f}%)")
print(f"  Variant {k}: All other patterns (remaining cases)")


# Assign variants using the patterns
w_samples_df = assign_variants_by_patterns(w_samples_df.drop(columns=['feature_pattern']), top_variants, k)

# Display variant distribution
variant_counts = w_samples_df['variant'].value_counts().sort_index()
print(f"\nFinal variant distribution:")
for variant, count in variant_counts.items():
    print(f"  Variant {variant}: {count} cases ({count/len(w_samples_df)*100:.2f}%)")

Assigning variants based on feature patterns...

Found 10 unique feature patterns
Top 10 patterns:
   1. Pattern 111111111111111111111111110000: 336 cases (82.96%)
   2. Pattern 111111111111111111111111100000: 32 cases (7.90%)
   3. Pattern 111111111111111111111111010000: 15 cases (3.70%)
   4. Pattern 111111111111111111111111000000: 7 cases (1.73%)
   5. Pattern 111111111111111111111111111100: 5 cases (1.23%)
   6. Pattern 111111111111111111111111111000: 4 cases (0.99%)
   7. Pattern 111111111111111111111110000000: 3 cases (0.74%)
   8. Pattern 111111111111111111111111110010: 1 cases (0.25%)
   9. Pattern 111111111111111111111110100000: 1 cases (0.25%)
  10. Pattern 111111111111111111111111110110: 1 cases (0.25%)

Using top 2 patterns as specific variants:
  Variant 1: 111111111111111111111111110000 (336 cases, 82.96%)
  Variant 2: 111111111111111111111111100000 (32 cases, 7.90%)
  Variant 3: All other patterns (remaining cases)
['InfectionSuspected', 'DiagnosticBlood', 'DisfuncOrg', 

In [8]:
# Create variant-specific dataframes
print("Creating variant-specific dataframes...")

variant_dataframes, variant_info = group_by_variants_with_filtered_columns(
    w_samples_df, num_variants=k
)

print(f"\nCreated {len(variant_dataframes)} variant-specific datasets:")
for variant_num in sorted(variant_dataframes.keys()):
    df = variant_dataframes[variant_num]
    info = variant_info[variant_info['variant'] == variant_num].iloc[0]
    print(f"  Variant {variant_num}:")
    print(f"    - Cases: {len(df)} ({info['percent']:.1f}%)")
    print(f"    - Features: {info['num_features']}")
    print(f"    - Pattern: {info['pattern']}")
    print(f"    - Columns: {df.columns.tolist()}")

# Display variant info table
print("\nVariant Information Summary:")
display(variant_info[['variant', 'count', 'percent', 'pattern', 'num_features', 'feature_columns']])

Creating variant-specific dataframes...

Created 3 variant-specific datasets:
  Variant 1:
    - Cases: 336 (83.0%)
    - Features: 26
    - Pattern: 111111111111111111111111110000
    - Columns: ['t', 'y', 'y0', 'y1', 'ite', 'InfectionSuspected', 'DiagnosticBlood', 'DisfuncOrg', 'SIRSCritTachypnea', 'Hypotensie', 'SIRSCritHeartRate', 'Infusion', 'DiagnosticArtAstrup', 'Age', 'DiagnosticIC', 'DiagnosticSputum', 'DiagnosticLiquor', 'DiagnosticOther', 'SIRSCriteria2OrMore', 'DiagnosticXthorax', 'SIRSCritTemperature', 'DiagnosticUrinaryCulture', 'SIRSCritLeucos', 'Oligurie', 'DiagnosticLacticAcid', 'Hypoxie', 'DiagnosticUrinarySediment', 'DiagnosticECG', 'Leucocytes', 'CRP', 'LacticAcid']
  Variant 2:
    - Cases: 32 (7.9%)
    - Features: 25
    - Pattern: 111111111111111111111111100000
    - Columns: ['t', 'y', 'y0', 'y1', 'ite', 'InfectionSuspected', 'DiagnosticBlood', 'DisfuncOrg', 'SIRSCritTachypnea', 'Hypotensie', 'SIRSCritHeartRate', 'Infusion', 'DiagnosticArtAstrup', 'Age', 'Diagn

,variant,count,percent,pattern,num_features,feature_columns
0,1,336,82.962963,111111111111111111111111110000,26,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
1,2,32,7.901235,111111111111111111111111100000,25,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
2,3,37,9.135802,111111111111111111111111010000,30,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
3,3,37,9.135802,111111111111111111111111111100,30,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
4,3,37,9.135802,111111111111111111111111111000,30,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
5,3,37,9.135802,111111111111111111111111110010,30,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
6,3,37,9.135802,111111111111111111111110000000,30,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
7,3,37,9.135802,111111111111111111111111000000,30,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
8,3,37,9.135802,111111111111111111111110100000,30,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."
9,3,37,9.135802,111111111111111111111111110110,30,"[InfectionSuspected, DiagnosticBlood, DisfuncO..."


## 6. Prepare Test and Validation Data

Extract test and validation datasets from the trained model and process them using the same variant patterns.

In [9]:
# Extract test data from the best model
print("Extracting test data from the trained model...")

w_test, t_test, y_test = best_model.get_data(dataset='test')
y_test_counterfactual = best_model.sample_y(
    t=1-t_test, w=w_test, ret_counterfactuals=False, seed=INITIAL_SEED
)

# Create test dataframe
if w_test.shape[1] == len(w_cols):
    w_test_df = pd.DataFrame(w_test, columns=w_cols)
else:
    column_names = [f"feature_{i}" for i in range(w_test.shape[1])]
    w_test_df = pd.DataFrame(w_test, columns=column_names)

w_test_df['t'] = t_test
w_test_df['y'] = y_test
w_test_df['y0'] = np.where(t_test.flatten() == 0, y_test.flatten(), y_test_counterfactual.flatten())
w_test_df['y1'] = np.where(t_test.flatten() == 1, y_test.flatten(), y_test_counterfactual.flatten())
w_test_df['ite'] = w_test_df['y1'] - w_test_df['y0']

print(f"Test data prepared:")
print(f"- Shape: {w_test_df.shape}")
print(f"- Treatment distribution: {dict(w_test_df['t'].value_counts())}")

# Extract training variant patterns for consistent assignment
training_variant_patterns = {}
for variant_num in sorted(list(variant_dataframes.keys())[:-1]):  # Exclude last variant
    training_pattern = variant_info[variant_info['variant'] == variant_num]['pattern'].iloc[0]
    training_variant_patterns[variant_num] = training_pattern

print(f"\nTraining variant patterns for consistent assignment:")
for variant_num, pattern in training_variant_patterns.items():
    print(f"  Variant {variant_num}: {pattern}")

Extracting test data from the trained model...
Test data prepared:
- Shape: (324, 35)
- Treatment distribution: {0.0: 297, 1.0: 27}

Training variant patterns for consistent assignment:
  Variant 1: 111111111111111111111111110000
  Variant 2: 111111111111111111111111100000


In [10]:
# Process test data using training patterns
print("Processing test data with training variant patterns...")

test_variant_dataframes, test_variant_info = process_test_data_with_training_variants(
    w_test_df, training_variant_patterns, num_variants=k
)

print(f"\nProcessed test data into {len(test_variant_dataframes)} variant-specific datasets:")
for variant_num in sorted(test_variant_dataframes.keys()):
    df = test_variant_dataframes[variant_num]
    print(f"  Test Variant {variant_num}: {len(df)} samples")

# Also create global test variant dataframes
global_test_variant_dataframes, global_test_variant_info = process_test_data_with_training_variants(
    w_test_df, training_variant_patterns, num_variants=len(training_variant_patterns)+1, for_global_model=True
)

print(f"\nCreated {len(global_test_variant_dataframes)} global test datasets for global model evaluation.")

Processing test data with training variant patterns...

Processed test data into 3 variant-specific datasets:
  Test Variant 1: 258 samples
  Test Variant 2: 33 samples
  Test Variant 3: 33 samples

Created 3 global test datasets for global model evaluation.


In [11]:
# Extract and process validation data
print("Preparing validation dataset...")

w_val, t_val, y_val = best_model.get_data(dataset='val')
y_val_counterfactual = best_model.sample_y(
    t=1-t_val, w=w_val, ret_counterfactuals=False, seed=INITIAL_SEED
)

# Create validation dataframe
if w_val.shape[1] == len(w_cols):
    w_val_df = pd.DataFrame(w_val, columns=w_cols)
else:
    column_names = [f"feature_{i}" for i in range(w_val.shape[1])]
    w_val_df = pd.DataFrame(w_val, columns=column_names)

w_val_df['t'] = t_val
w_val_df['y'] = y_val
w_val_df['y0'] = np.where(t_val.flatten() == 0, y_val.flatten(), y_val_counterfactual.flatten())
w_val_df['y1'] = np.where(t_val.flatten() == 1, y_val.flatten(), y_val_counterfactual.flatten())
w_val_df['ite'] = w_val_df['y1'] - w_val_df['y0']

# Process validation data into variants
val_variant_dataframes, val_variant_info = process_test_data_with_training_variants(
    w_val_df, training_variant_patterns, num_variants=k
)

global_val_variant_dataframes, global_val_variant_info = process_test_data_with_training_variants(
    w_val_df, training_variant_patterns, num_variants=len(training_variant_patterns)+1, for_global_model=True
)

print(f"Validation data prepared:")
print(f"- Shape: {w_val_df.shape}")
print(f"- Treatment distribution: {dict(w_val_df['t'].value_counts())}")
print(f"- Variant-specific datasets: {len(val_variant_dataframes)}")
print(f"- Global validation datasets: {len(global_val_variant_dataframes)}")

print("\nValidation set sizes by variant:")
for variant_num in sorted(val_variant_dataframes.keys()):
    print(f"  Variant {variant_num}: {len(val_variant_dataframes[variant_num])} samples")

print("\nData preparation completed!")
print("Ready for causal inference experiments.")

Preparing validation dataset...
Validation data prepared:
- Shape: (81, 35)
- Treatment distribution: {0.0: 72, 1.0: 9}
- Variant-specific datasets: 3
- Global validation datasets: 3

Validation set sizes by variant:
  Variant 1: 67 samples
  Variant 2: 7 samples
  Variant 3: 7 samples

Data preparation completed!
Ready for causal inference experiments.


## 7. Multi-Seed Causal Inference Experiments

Run comprehensive causal inference experiments across multiple random seeds to assess the robustness of both global and variant-specific modeling approaches.

### 7.1 Experiment Configuration

Configure the multi-seed experiment parameters.

In [12]:
# Configure multi-seed experiment
experiment_seeds = [0, 1]  # Can expand to more seeds. In our paper: list(range(0, 500))
results_save_path = "C:\\Users\\Guy\\OneDrive\\Shared Documents\\Technion\\Msc\\process_mining_BP_CDM\\thesis\\code\\realcause-for-cpvs\\cdv_experiments_results_example.pkl"

print("Multi-Seed Experiment Configuration:")
print("=" * 50)
print(f"Experiment seeds: {experiment_seeds}")
print(f"Number of seeds: {len(experiment_seeds)}")
print(f"R² threshold for model selection: {R2_THRESHOLD}")
print(f"Number of variants: {NUM_VARIANTS}")
print(f"Results save path: {results_save_path}")

print(f"\nExperiment will compare:")
print(f"1. Global Method: Single model trained on all data")
print(f"2. Variant Method: Separate models per variant (with global fallback)")

print(f"\nFor each method, we will evaluate:")
print(f"- All estimator types (S-Learner, T-Learner, X-Learner, DR-Learner, Double-ML)")
print(f"- Best estimator selection based on validation performance")

print(f"\nReady to start experiment...")

Multi-Seed Experiment Configuration:
Experiment seeds: [0, 1]
Number of seeds: 2
R² threshold for model selection: 0.1
Number of variants: 3
Results save path: C:\Users\Guy\OneDrive\Shared Documents\Technion\Msc\process_mining_BP_CDM\thesis\code\realcause-for-cpvs\cdv_experiments_results_example.pkl

Experiment will compare:
1. Global Method: Single model trained on all data
2. Variant Method: Separate models per variant (with global fallback)

For each method, we will evaluate:
- All estimator types (S-Learner, T-Learner, X-Learner, DR-Learner, Double-ML)
- Best estimator selection based on validation performance

Ready to start experiment...


### 7.2 Run Multi-Seed Experiment

Execute the main experiment loop across all seeds.

In [13]:
# Force reload modules to get the updated tqdm imports
import importlib
import cdv_utils.experiment_runner
importlib.reload(cdv_utils.experiment_runner)
from cdv_utils.experiment_runner import run_multi_seed_experiment

# Run the multi-seed experiment
print("Starting Multi-Seed Causal Inference Experiment...")
print("This may take several minutes depending on the number of seeds and data size.")
print("\n" + "=" * 70)

try:
    results_by_seed = run_multi_seed_experiment(
        best_model=best_model,
        experiment_seeds=experiment_seeds,
        w_cols=w_cols,
        top_variants=top_variants,
        k=k,
        training_variant_patterns=training_variant_patterns,
        test_variant_dataframes=test_variant_dataframes,
        val_variant_dataframes=val_variant_dataframes,
        global_test_variant_dataframes=global_test_variant_dataframes,
        global_val_variant_dataframes=global_val_variant_dataframes,
        results_save_path=results_save_path,
        initial_seed=INITIAL_SEED,
        r2_threshold=R2_THRESHOLD
    )
    
    print("\n" + "=" * 70)
    print("Multi-Seed Experiment COMPLETED Successfully!")
    print("=" * 70)
    
except Exception as e:
    print(f"\n❌ Experiment failed with error: {str(e)}")
    import traceback
    traceback.print_exc()
    
    # Try to load partial results
    try:
        from cdv_utils.experiment_runner import load_experiment_results
        results_by_seed = load_experiment_results(results_save_path)
        if results_by_seed:
            print(f"\n✅ Loaded {len(results_by_seed)} partial results from {results_save_path}")
        else:
            print(f"\n❌ No partial results found.")
            results_by_seed = {}
    except:
        results_by_seed = {}

Starting Multi-Seed Causal Inference Experiment...
This may take several minutes depending on the number of seeds and data size.

Starting Multi-Seed Experiment with 2 seeds


Processing seeds:   0%|          | 0/2 [00:00<?, ?it/s]


SEED 0 (1/2)
[0] Step 1/5: Generating training data...
['InfectionSuspected', 'DiagnosticBlood', 'DisfuncOrg', 'SIRSCritTachypnea', 'Hypotensie', 'SIRSCritHeartRate', 'Infusion', 'DiagnosticArtAstrup', 'Age', 'DiagnosticIC', 'DiagnosticSputum', 'DiagnosticLiquor', 'DiagnosticOther', 'SIRSCriteria2OrMore', 'DiagnosticXthorax', 'SIRSCritTemperature', 'DiagnosticUrinaryCulture', 'SIRSCritLeucos', 'Oligurie', 'DiagnosticLacticAcid', 'Hypoxie', 'DiagnosticUrinarySediment', 'DiagnosticECG', 'Leucocytes', 'CRP', 'LacticAcid', 'Leucocytes_2', 'CRP_2', 'LacticAcid_2', 'LacticAcid_3']
[0] Step 2/5: Initializing estimators...
[0] Step 3/5: Training models and generating predictions...
[0] Step 4/5: Predicting on validation data for best model selection...


Processing seeds:  50%|█████     | 1/2 [00:13<00:13, 13.82s/it]

[0] Selecting best global model...
[0] Selecting best models per variant...
[0] Step 5/5: Creating result dataframes with best models...
[0] ✓ Completed. Results saved to C:\Users\Guy\OneDrive\Shared Documents\Technion\Msc\process_mining_BP_CDM\thesis\code\realcause-for-cpvs\cdv_experiments_results_example.pkl

SEED 1 (2/2)
[1] Step 1/5: Generating training data...
['InfectionSuspected', 'DiagnosticBlood', 'DisfuncOrg', 'SIRSCritTachypnea', 'Hypotensie', 'SIRSCritHeartRate', 'Infusion', 'DiagnosticArtAstrup', 'Age', 'DiagnosticIC', 'DiagnosticSputum', 'DiagnosticLiquor', 'DiagnosticOther', 'SIRSCriteria2OrMore', 'DiagnosticXthorax', 'SIRSCritTemperature', 'DiagnosticUrinaryCulture', 'SIRSCritLeucos', 'Oligurie', 'DiagnosticLacticAcid', 'Hypoxie', 'DiagnosticUrinarySediment', 'DiagnosticECG', 'Leucocytes', 'CRP', 'LacticAcid', 'Leucocytes_2', 'CRP_2', 'LacticAcid_2', 'LacticAcid_3']
[1] Step 2/5: Initializing estimators...
[1] Step 3/5: Training models and generating predictions...
[1] 

Processing seeds: 100%|██████████| 2/2 [00:27<00:00, 13.70s/it]

[1] Selecting best global model...
[1] Selecting best models per variant...
[1] Step 5/5: Creating result dataframes with best models...
[1] ✓ Completed. Results saved to C:\Users\Guy\OneDrive\Shared Documents\Technion\Msc\process_mining_BP_CDM\thesis\code\realcause-for-cpvs\cdv_experiments_results_example.pkl

Multi-Seed Experiment Complete!
Total seeds processed: 2
Results saved to: C:\Users\Guy\OneDrive\Shared Documents\Technion\Msc\process_mining_BP_CDM\thesis\code\realcause-for-cpvs\cdv_experiments_results_example.pkl

Multi-Seed Experiment COMPLETED Successfully!


### 7.3 Analyze Experiment Results

Analyze and summarize the results from the multi-seed experiment.

In [14]:
# Analyze experiment results
if results_by_seed:
    print("Analyzing Multi-Seed Experiment Results...")
    print("=" * 50)
    
    # Get analysis summary
    analysis = analyze_experiment_results(results_by_seed)
    
    print(f"\nExperiment Summary:")
    print(f"- Total seeds attempted: {analysis['total_seeds']}")
    print(f"- Successfully completed: {len(analysis['successful_seeds'])}")
    print(f"- Failed: {len(analysis['failed_seeds'])}")
    
    if analysis['successful_seeds']:
        print(f"- Success rate: {len(analysis['successful_seeds'])/analysis['total_seeds']*100:.1f}%")
        
        print(f"\nSuccessful seeds: {analysis['successful_seeds']}")
        if analysis['failed_seeds']:
            print(f"Failed seeds: {analysis['failed_seeds']}")
        
        # Show detailed results for each successful seed
        print(f"\nDetailed Results by Seed:")
        for seed in analysis['successful_seeds']:
            summary = analysis['summary_by_seed'][seed]
            print(f"\n  Seed {seed}:")
            print(f"    - Global method (all estimators): {summary['global_method_shape']}")
            print(f"    - Global method (best): {summary['global_method_best_shape']}")
            print(f"    - Variant method (all estimators): {summary['variant_method_shape']}")
            print(f"    - Variant method (best): {summary['variant_method_best_shape']}")
            
            if summary['best_global_model']:
                bgm = summary['best_global_model']
                print(f"    - Best global model: {bgm['estimator']} (R²: {bgm['r2']:.4f}, ATE bias: {bgm['ate_bias']:.6f})")
            
            print(f"    - Best models selected for {summary['best_models_count']} variants")
    else:
        print("\n❌ No successful experiments to analyze.")
        print("Please check the error messages above and retry with different parameters.")
        
else:
    print("❌ No experiment results available to analyze.")
    print("The experiment may have failed or no results were saved.")

Analyzing Multi-Seed Experiment Results...

Experiment Summary:
- Total seeds attempted: 2
- Successfully completed: 2
- Failed: 0
- Success rate: 100.0%

Successful seeds: [0, 1]

Detailed Results by Seed:

  Seed 0:
    - Global method (all estimators): (1944, 11)
    - Global method (best): (324, 11)
    - Variant method (all estimators): (1944, 12)
    - Variant method (best): (324, 12)
    - Best global model: S-Learner (Linear) (R²: -0.0140, ATE bias: 1422.668820)
    - Best models selected for 3 variants

  Seed 1:
    - Global method (all estimators): (1944, 11)
    - Global method (best): (324, 11)
    - Variant method (all estimators): (1944, 12)
    - Variant method (best): (324, 12)
    - Best global model: S-Learner (Linear) (R²: -0.0002, ATE bias: 150.766137)
    - Best models selected for 3 variants


### 7.4 Results Summary and Export

Provide final summary and export key results for further analysis.

In [15]:
# Final results summary
if results_by_seed and len(results_by_seed) > 0:
    print("Final Results Summary")
    print("=" * 50)
    
    # Get a sample of results to show structure
    first_seed = list(results_by_seed.keys())[0]
    sample_results = results_by_seed[first_seed]
    
    print(f"\nResults Structure (example from seed {first_seed}):")
    print(f"\n1. Global Method (All Estimators):")
    if not sample_results['global_method'].empty:
        gm_df = sample_results['global_method']
        print(f"   - Shape: {gm_df.shape}")
        print(f"   - Variants: {sorted(gm_df['variant'].unique())}")
        print(f"   - Estimators: {sorted(gm_df['estimator'].unique())}")
        display(gm_df.head(3))
    
    print(f"\n2. Variant Method (Best Estimators):")
    if not sample_results['variant_method_best'].empty:
        vmb_df = sample_results['variant_method_best']
        print(f"   - Shape: {vmb_df.shape}")
        print(f"   - Variants: {sorted(vmb_df['variant'].unique())}")
        print(f"   - Estimators: {sorted(vmb_df['estimator'].unique())}")
        display(vmb_df.head(3))
    
    # Export summary statistics
    export_path = "results/experiment_summary.json"
    
    try:
        # Create exportable summary
        export_summary = {
            'experiment_config': {
                'seeds': experiment_seeds,
                'num_variants': NUM_VARIANTS,
                'r2_threshold': R2_THRESHOLD,
                'initial_seed': INITIAL_SEED
            },
            'results': analysis
        }
        
        with open(export_path, 'w') as f:
            json.dump(export_summary, f, indent=2, default=str)
        
        print(f"\n✅ Summary exported to: {export_path}")
        
    except Exception as e:
        print(f"\n⚠️ Could not export summary: {str(e)}")
    
    print(f"\n✅ Complete results saved to: {results_save_path}")
    print(f"\nTo load results in future sessions:")
    print(f"```python")
    print(f"from cdv_utils.experiment_runner import load_experiment_results")
    print(f"results = load_experiment_results('{results_save_path}')")
    print(f"```")
    
else:
    print("\n❌ No results to summarize.")
    print("\nTo retry the experiment:")
    print("1. Check that all data preparation steps completed successfully")
    print("2. Verify that the RealCause model was trained properly")
    print("3. Consider using fewer seeds or simpler model configurations for initial testing")

print("\n" + "=" * 70)
print("CDV Modeling Notebook Execution Complete!")
print("=" * 70)

Final Results Summary

Results Structure (example from seed 0):

1. Global Method (All Estimators):
   - Shape: (1944, 11)
   - Variants: [1, 2, 3]
   - Estimators: ['DR Learner (EconML)', 'Double ML', 'S-Learner (Linear)', 'S-Learner (RF)', 'T-Learner (RF)', 'X-Learner (RF)']


,variant,estimator,method,t,y,y0_real,y1_real,y0_pred,y1_pred,ite_real,ite_pred
0,1,S-Learner (Linear),global,0.0,10886.0,10886.0,45481.902344,7787.081335,5791.162805,34595.902344,-1995.91853
1,1,S-Learner (Linear),global,0.0,4477.0,4477.0,6704.216797,7787.081335,5791.162805,2227.216797,-1995.91853
2,1,S-Learner (Linear),global,0.0,14947.0,14947.0,2850.990479,7787.081335,5791.162805,-12096.009521,-1995.91853



2. Variant Method (Best Estimators):
   - Shape: (324, 12)
   - Variants: [1, 2, 3]
   - Estimators: ['S-Learner (Linear)', 'best_global_model']


,variant,estimator,method,used_global_model,t,y,y0_real,y1_real,y0_pred,y1_pred,ite_real,ite_pred
0,1,S-Learner (Linear),variant,False,0.0,10886.0,10886.0,45481.902344,7448.014388,6081.399674,34595.902344,-1366.614714
1,1,S-Learner (Linear),variant,False,0.0,4477.0,4477.0,6704.216797,7448.014388,6081.399674,2227.216797,-1366.614714
2,1,S-Learner (Linear),variant,False,0.0,14947.0,14947.0,2850.990479,7448.014388,6081.399674,-12096.009521,-1366.614714



✅ Summary exported to: results/experiment_summary.json

✅ Complete results saved to: C:\Users\Guy\OneDrive\Shared Documents\Technion\Msc\process_mining_BP_CDM\thesis\code\realcause-for-cpvs\cdv_experiments_results_example.pkl

To load results in future sessions:
```python
from cdv_utils.experiment_runner import load_experiment_results
results = load_experiment_results('C:\Users\Guy\OneDrive\Shared Documents\Technion\Msc\process_mining_BP_CDM\thesis\code\realcause-for-cpvs\cdv_experiments_results_example.pkl')
```

CDV Modeling Notebook Execution Complete!
